In [208]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.remote.webelement import WebElement

import string
import random
import time

In [114]:
## characters to generate password from
characters = list(string.ascii_letters + string.digits)

def generate_random_password(length):
	random.shuffle(characters)
	password = []
	for i in range(length):
		password.append(random.choice(characters))
	password.append("!")
	return "".join(password)

def get_proxies():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options)
    driver.get("https://sslproxies.org/")
    driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered']//th[contains(., 'IP Address')]"))))
    ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 1]")))]
    ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 2]")))]
    driver.quit()
    proxies = []
    for i in range(0, len(ips)):
        proxies.append(ips[i]+':'+ports[i])
    return proxies


In [222]:
class Driver:
    def __init__(self, proxy = None):
        options = webdriver.ChromeOptions()
        # options.add_argument("start-maximized")
        options.add_experimental_option("prefs", { \
            "profile.default_content_setting_values.notifications": 1 
            })

        if proxy is not None:
            options.add_argument('--proxy-server={}'.format(proxy))
        self.driver = webdriver.Chrome(options=options)
    
    def login(self, user, password):
        self.driver.get("https://www.reddit.com/")
        try: 
            WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//a[starts-with(@href, 'https://www.reddit.com/login')]"))).click()
        except TimeoutException:
            try: 
                time.sleep(3)   
                self.driver.refresh()
                WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//a[starts-with(@href, 'https://www.reddit.com/login')]"))).click()
            except TimeoutException:
                time.sleep(3)   
                self.driver.refresh()
                WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//a[starts-with(@href, 'https://www.reddit.com/login')]"))).click() 
        try:
            WebDriverWait(self.driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[starts-with(@src, 'https://www.reddit.com/login')]")))
        except TimeoutException:
            pass 
        time.sleep(1)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='loginUsername']"))).send_keys(user)
        self.driver.find_element(By.XPATH, "//input[@id='loginPassword']").send_keys(password)
        self.driver.find_element(By.XPATH, "//button[@class='AnimatedForm__submitButton m-full-width']").click()
        
    def change_email_and_password(self, password, new_password, new_email):
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@name='createPost']")))
        self.driver.get("https://www.reddit.com/settings/")
        # Change email
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='_2iuoyPiKHN3kfOoeIQalDT _2tU8R9NTqhvBrhoNAXWWcP HNozj_dKjQZ59ZsfEegz8 ']"))).send_keys(password)[0].click()
        # self.driver.find_elements(By.XPATH, "//*[@class='_2iuoyPiKHN3kfOoeIQalDT _2tU8R9NTqhvBrhoNAXWWcP HNozj_dKjQZ59ZsfEegz8 ']")[0].click()
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@type='password']"))).send_keys(password)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@placeholder='New email']"))).send_keys(new_email)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@type='submit']"))).click()
        # Switch to default
        self.driver.switch_to.default_content()
        # Change password
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='_2iuoyPiKHN3kfOoeIQalDT _2tU8R9NTqhvBrhoNAXWWcP HNozj_dKjQZ59ZsfEegz8 ']"))).send_keys(password)[1].click()
        # self.driver.find_elements(By.XPATH, "//*[@class='_2iuoyPiKHN3kfOoeIQalDT _2tU8R9NTqhvBrhoNAXWWcP HNozj_dKjQZ59ZsfEegz8 ']")[1].click()
        WebDriverWait(self.driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[starts-with(@src, 'https://www.reddit.com/change_password')]")))
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='old_password']"))).send_keys(password)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='password']"))).send_keys(new_password)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@id='password2']"))).send_keys(new_password)
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@class='AnimatedForm__submitButton']"))).click()
        return new_password
    
    def close(self):
        self.driver.close()
    
    def api_prefs(self, user):
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@name='createPost']")))
        self.driver.get("https://www.reddit.com/prefs/apps")
        # WebDriverWait(self.driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id = 'gtm-jail')]")))
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@id= 'create-app-button']"))).click() 
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@name= 'name']"))).send_keys(user)
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@id= 'app_type_script']"))).click()
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@name= 'redirect_uri']"))).send_keys("https://www.google.com")
        WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@type= 'submit']"))).click() 
        # Get API info
        client_id = WebDriverWait(chrome.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='app-details']/h3[2]")))
        client_secret = WebDriverWait(chrome.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//td[@class='prefright']")))
        return client_id.text, client_secret.text

In [146]:
users = []
with open('accounts.csv') as f:
    for line in f:
        users.append(line.strip())
        
user_index = 0


In [191]:
count = 0
proxy_list = ["20.47.108.204:8888"]
while user_index < len(users):
  userpass = users[user_index]
  # proxy = proxies[index % len(proxies)]
  print(proxy)
  proxy = proxy_list[user_index % len(proxy_list)]
  chrome = Driver(proxy)
  user, password = get_user_pass(userpass)  
  chrome.login(user, password)
  new_password = generate_random_password(10)
  new_email = user + "@gmail.com"
  chrome.change_email_and_password(password, new_password, new_email)
  print(user + new_password)
  chrome.close()
  out = open('newaccounts_withapi.csv', 'a+')
  out.write(user+":"+new_password + ":" + proxy + "\n")
  out.close()
  count+=1
  user_index+=1
  time.sleep(4)
  

20.47.108.204:8888


KeyboardInterrupt: 

In [221]:
chrome = Driver()
chrome.login("Known_Grade_1184", "ruth#N9s")
print(chrome.api_prefs("Known_Grade_1184"))
# # Change email
# self.driver.find_elements(By.XPATH, "//*[@class='_2iuoyPiKHN3kfOoeIQalDT _2tU8R9NTqhvBrhoNAXWWcP HNozj_dKjQZ59ZsfEegz8 ']")[0].click()
# WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@type='password']"))).send_keys(password)
# WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//input[@placeholder='New email']"))).send_keys(new_email)
# WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@type='submit']"))).click()

(<selenium.webdriver.remote.webelement.WebElement (session="aeeeaa194f19d9426f4d446de6e9084d", element="b98001f7-1827-485a-8585-a0e48d9a6b8c")>, <selenium.webdriver.remote.webelement.WebElement (session="aeeeaa194f19d9426f4d446de6e9084d", element="7853a9e1-e510-4ef9-89b1-c71536d70ea6")>)


In [218]:
client_id = WebDriverWait(chrome.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='app-details']/h3[2]")))
print(client_id.text)
client_secret = WebDriverWait(chrome.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//td[@class='prefright']")))
print(client_secret.text)

zxsubK92KO2rbLs3xJCmdw
npU7DyVtLo2_IgV2Akwf09rjdT0b6Q
